In [1]:
import pandas as pd
import numpy as np
import re

df=pd.read_csv('glassdoor_data.csv')
df=df[df['Salary Estimate']!='-1']

df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'Employer est.' in x.lower() else 0)
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0].replace('K','').replace('$',''))
# salary=minusk.apply(lambda x: x.replace('Per Hour','').replace('Employer est.',''))

# Extract minimum and maximum salary from "Salary Estimate" column
def extract_min_salary(salary):
    try:
        split_values = salary.split('-')
        if len(split_values) >= 2:
            return float(split_values[0].strip().lstrip('$').rstrip('K').rstrip(' per hour'))
    except:
        pass
    return None

def extract_max_salary(salary):
    try:
        split_values = salary.split('-')
        if len(split_values) >= 2:
            return float(split_values[1].split('(')[0].strip().lstrip('$').rstrip('K').rstrip(' per hour'))
    except:
        pass
    return None

df['Min Salary'] = salary.apply(extract_min_salary)
df['Max Salary'] = salary.apply(extract_max_salary)
df['Rating'] = df['Rating'].apply(lambda x: x.split(' ')[0])
# df['Company Name'] = df['Company Name'].apply(lambda x: x.split(' ')[0])

def remove_numerical_part(value):
    value = re.sub(r'\d+', '', value)  # Remove numerical part
    value = re.sub(r'[^\w\s]', '', value)  # Remove special characters
    return value.strip()  # Remove leading and trailing whitespaces


df['Company Name'] = df['Company Name'].apply(remove_numerical_part)

def extract_jobstate(location):
    splitval = location.split(',')
    if len(splitval)>=2:
        return (splitval[1])

df['job_state'] = df['Location'].apply(extract_jobstate)

# df.job_state.value_counts()

df['age']=df['Founded'].apply(lambda x: x if x<1 else (2023-int(x)))

df['python']=df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

df['excel']=df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

df['sql']=df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)

df.sql.value_counts()

def extract_company(ownership):
    splitval=ownership.split('-')
    if len(splitval)>1:
        return splitval[1]
df['Type of ownership']=df['Type of ownership'].apply(extract_company)

# df.columns
df_out = df.drop(['Unnamed: 0','Headquarters','Competitors'], axis =1)

df_out.to_csv('data_cleaned.csv',index=False)

In [2]:
df_out

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,hourly,employer_provided,Min Salary,Max Salary,job_state,age,python,excel,sql
0,Data Analyst I,$41K - $55K (Glassdoor est.),Job Summary:\nIf you live in or around the San...,4.3,Firstmark Credit Union,"San Antonio, TX",201 to 500 Employees,1932,Private,Banking & Lending,...,$1 to $5 billion (USD),0,0,41.0,55.0,TX,91,0,0,0
1,Reporting Data Analyst,$64K - $92K (Glassdoor est.),"Medlogix, LLC delivers innovative medical clai...",2.6,MEDLOGIX LLC,"Lansdale, PA",Unknown,-1,Private,Insurance Carriers,...,Unknown / Non-Applicable,0,0,64.0,92.0,PA,-1,0,0,0
2,Junior Data Analyst,$45K - $69K (Glassdoor est.),Hintz and Co is a boutique full service media ...,3.8,Hintz and Co,"Los Angeles, CA",1 to 50 Employees,1979,Private,Architectural & Engineering Services,...,$5 to $25 million (USD),0,0,45.0,69.0,CA,44,0,0,0
3,Data Analyst(SQL),$64K - $106K (Glassdoor est.),Job Title: Data Analyst (SQL)\nJob Summary: We...,4.1,Leading Torch LLC,"Plano, TX",1 to 50 Employees,-1,Private,-1,...,Less than $1 million (USD),0,0,64.0,106.0,TX,-1,0,0,1
4,Data Analyst (Jr.),$80K - $100K (Employer est.),Department of Veterans Affairs experience? Do ...,4.0,Ben Allegretti Consulting BAC,"Washington, DC",Unknown,2007,None,National Agencies,...,Unknown / Non-Applicable,0,0,80.0,100.0,DC,16,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Data Analyst,$49K - $70K (Glassdoor est.),Chicago Fire Football Club is a Major League S...,3.8,Chicago Fire FC,"Bridgeview, IL",51 to 200 Employees,-1,Private,Sports & Recreation,...,$5 to $25 million (USD),0,0,49.0,70.0,IL,-1,0,0,0
196,Junior Data Analyst,$59K - $95K (Employer est.),Junior Data Analyst\nThis role is open to our ...,4.1,Hotwire Global,"San Francisco, CA",201 to 500 Employees,2000,Public,Advertising & Public Relations,...,Unknown / Non-Applicable,0,0,59.0,95.0,CA,23,0,0,0
197,Junior Business Analyst,$65K - $90K (Employer est.),Who We Are: ThinkTek is a fast-growing Certifi...,5.0,ThinkTek LLC,"Fairfax, VA",1 to 50 Employees,-1,Private,-1,...,Unknown / Non-Applicable,0,0,65.0,90.0,VA,-1,0,0,0
198,Data Analyst,$67K - $100K (Glassdoor est.),Aura is on a mission to create a safer interne...,4.3,Aura,"Boston, MA",201 to 500 Employees,2017,Private,Software Development,...,$100 to $500 million (USD),0,0,67.0,100.0,MA,6,0,0,0
